In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sqlite3 as sql

In [ ]:
# open database, import data into pandas
conn = sql.connect("sbtb.db")
df = pd.read_sql_query("SELECT * from Reviews;", conn)

print(df.head())

conn.close()

In [ ]:
reviewers = df[['Reviewer','Id']]
reviewer_count = reviewers.groupby('Reviewer').count()
print(reviewer_count)

In [ ]:
reviewer_count.plot(kind='bar')